In [79]:
from lxml import etree
from datetime import timedelta, datetime
import pytz

In [92]:
xml_path = 'in/input-test-script.xml'
stylesheet = 'in/stylesheet.xsl'



In [99]:
# load stylesheet
xslt_doc = etree.parse(stylesheet)
xslt_transform = etree.XSLT(xslt_doc)

In [100]:
# load input doc
input_doc = etree.parse(xml_path)

In [101]:
import pandas as pd

In [102]:
df = pd.read_xml(xml_path, stylesheet=stylesheet)

In [103]:
df #ahuevooooo puta madreeeeeeee

,ServicePointID,MeterID,ChannelID,Timestamp,IntervalValue,Usage,EnergyType,EnergyUnits,StatusRef,IsEstimateOrOutageCodes,IsDeleted,UpdatedDatetime,IsEstimate,IsOutage
0,102976573,102976573,1,2023-04-04T04:00:00Z,15,754.000,Electric,kWh,58,"NV,VEESET1PV,VEESET2NV,VEESET3NV",False,2023-04-05T07:01:20Z,True,True
1,102976573,102976573,1,2023-04-04T04:00:00Z,15,255.000,Electric,kWh,58,"NV,VEESET1PV,VEESET2NV,VEESET3NV",False,2023-04-05T07:01:20Z,True,True
2,102976573,102976573,1,2023-04-04T04:00:00Z,15,336.000,Electric,kWh,58,"NV,VEESET1PV,VEESET2NV,VEESET3NV",False,2023-04-05T07:01:20Z,True,True
3,102976527,102976527,1,2023-04-04T04:00:00Z,15,49.000,Electric,kWh,58,"NV,VEESET1PV,VEESET2NV,VEESET3NV",False,2023-04-05T07:01:20Z,True,True
4,102976527,102976527,1,2023-04-04T04:00:00Z,15,56.000,Electric,kWh,58,"NV,VEESET1PV,VEESET2NV,VEESET3NV",False,2023-04-05T07:01:20Z,True,True
5,102976527,102976527,1,2023-04-04T04:00:00Z,15,80.000,Electric,kWh,58,"NV,VEESET1PV,VEESET2NV,VEESET3NV",False,2023-04-05T07:01:20Z,True,True
6,102976527,102976527,1,2023-04-04T04:00:00Z,15,75.000,Electric,kWh,58,"NV,VEESET1PV,VEESET2NV,VEESET3NV",False,2023-04-05T07:01:20Z,True,True
7,102976527,102976527,1,2023-04-04T04:00:00Z,15,69.000,Electric,kWh,58,"NV,VEESET1PV,VEESET2NV,VEESET3NV",False,2023-04-05T07:01:20Z,True,True
8,102976496,102976496,1,2023-04-04T04:00:00Z,15,72.000,Electric,kWh,58,"NV,VEESET1PV,VEESET2NV,VEESET3NV",False,2023-04-05T07:01:20Z,True,True
9,102976496,102976496,1,2023-04-04T04:00:00Z,15,55.000,Electric,kWh,58,"NV,VEESET1PV,VEESET2NV,VEESET3NV",False,2023-04-05T07:01:20Z,True,True


In [104]:
# implement business rules

# energy type mapping
energy_type = {
    'Electric': 'E',
    'Gas': 'G',
    'Water': 'W'
}

# energy direction mapping
energy_direction = {
    101: 'delivered',
    1: 'delivered',
    102: 'received',
    2: 'received'
}


df.assign(
    Usage = 0.001 * df['Usage'],
    Timestamp = (pd.to_datetime(df['Timestamp']).apply(lambda t:  t.astimezone(pytz.timezone('US/Eastern'))) + pd.to_timedelta(df['IntervalValue'], 'minutes')
).dt.strftime("%Y-%m-%dT%H:%M:%S.000%z").replace('-0400', '-04:00').replace('-0500', '-05:00'),
    Timezone = (pd.to_datetime(df['Timestamp']).apply(lambda t:  t.astimezone(pytz.timezone('US/Eastern'))) + pd.to_timedelta(df['IntervalValue'], 'minutes')
).dt.strftime("%Z"),
    IntervalUnits = 'minute',
    EnergyType = df['EnergyType'].apply(lambda x: energy_type.get(x, 'E')),
    EnergyDirection = df['ChannelID'].apply(lambda x: energy_direction.get(x, 'CHANNEL ID NOT FOUND')),
    ChannelID = df['ChannelID'].apply(lambda x: 'usage_' + energy_direction.get(x, 'CHANNEL ID NOT FOUND'))
)

,ServicePointID,MeterID,ChannelID,Timestamp,IntervalValue,Usage,EnergyType,EnergyUnits,StatusRef,IsEstimateOrOutageCodes,IsDeleted,UpdatedDatetime,IsEstimate,IsOutage,Timezone,IntervalUnits,EnergyDirection
0,102976573,102976573,usage_delivered,2023-04-04T00:15:00.000-0400,15,0.754000,E,kWh,58,"NV,VEESET1PV,VEESET2NV,VEESET3NV",False,2023-04-05T07:01:20Z,True,True,EDT,minute,delivered
1,102976573,102976573,usage_delivered,2023-04-04T00:15:00.000-0400,15,0.255000,E,kWh,58,"NV,VEESET1PV,VEESET2NV,VEESET3NV",False,2023-04-05T07:01:20Z,True,True,EDT,minute,delivered
2,102976573,102976573,usage_delivered,2023-04-04T00:15:00.000-0400,15,0.336000,E,kWh,58,"NV,VEESET1PV,VEESET2NV,VEESET3NV",False,2023-04-05T07:01:20Z,True,True,EDT,minute,delivered
3,102976527,102976527,usage_delivered,2023-04-04T00:15:00.000-0400,15,0.049000,E,kWh,58,"NV,VEESET1PV,VEESET2NV,VEESET3NV",False,2023-04-05T07:01:20Z,True,True,EDT,minute,delivered
4,102976527,102976527,usage_delivered,2023-04-04T00:15:00.000-0400,15,0.056000,E,kWh,58,"NV,VEESET1PV,VEESET2NV,VEESET3NV",False,2023-04-05T07:01:20Z,True,True,EDT,minute,delivered
5,102976527,102976527,usage_delivered,2023-04-04T00:15:00.000-0400,15,0.080000,E,kWh,58,"NV,VEESET1PV,VEESET2NV,VEESET3NV",False,2023-04-05T07:01:20Z,True,True,EDT,minute,delivered
6,102976527,102976527,usage_delivered,2023-04-04T00:15:00.000-0400,15,0.075000,E,kWh,58,"NV,VEESET1PV,VEESET2NV,VEESET3NV",False,2023-04-05T07:01:20Z,True,True,EDT,minute,delivered
7,102976527,102976527,usage_delivered,2023-04-04T00:15:00.000-0400,15,0.069000,E,kWh,58,"NV,VEESET1PV,VEESET2NV,VEESET3NV",False,2023-04-05T07:01:20Z,True,True,EDT,minute,delivered
8,102976496,102976496,usage_delivered,2023-04-04T00:15:00.000-0400,15,0.072000,E,kWh,58,"NV,VEESET1PV,VEESET2NV,VEESET3NV",False,2023-04-05T07:01:20Z,True,True,EDT,minute,delivered
9,102976496,102976496,usage_delivered,2023-04-04T00:15:00.000-0400,15,0.055000,E,kWh,58,"NV,VEESET1PV,VEESET2NV,VEESET3NV",False,2023-04-05T07:01:20Z,True,True,EDT,minute,delivered
